# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [16]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import torch.optim as optim
import time

BATCH_SIZE = 128
NUM_EPOCHS = 20

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Processing...
Done!


Then, we define the model, object function and optimizer that we use to classify.

In [ ]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.layer1 = nn.Sequential(

            # 1-1 conv layer
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(),

            # 1-2 conv layer
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(),

            # 1 Pooling layer
            nn.AvgPool2d(kernel_size=2, stride=2))

        self.layer2 = nn.Sequential(

            # 1-1 conv layer
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(),

            # 1-2 conv layer
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            
            nn.BatchNorm2d(64),
            nn.LeakyReLU(),

            # 1 Pooling layer
            nn.AvgPool2d(kernel_size=2, stride=2))

        self.layer3 = nn.Sequential(
            
            nn.Linear(3136, 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(),

            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(),

            nn.Linear(64, 10))

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        features = out.view(out.shape[0], -1)
        out = self.layer3(features)
        return out

if torch.cuda.is_available():
    model = SimpleNet().cuda()

criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

Next, we can start to train and evaluate!

In [34]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    correct = 0
    total = 0
    total_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        if torch.cuda.is_available():
            data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)

        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        total += target.shape[0]

        loss = criterion(output, target)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    log = ('Train Epoch: {}\tAcc:[{}/{}]{:.6f} Loss: {:.6f}'.format(
                epoch, int(correct), int(total), float(correct)/float(total)*100, total_loss))
    print(log)
    

correct = 0
total = 0
total_loss = 0
for batch_idx, (data, target) in enumerate(test_loader):
    if torch.cuda.is_available():
        data, target = data.cuda(), target.cuda()
    output = model(data)
    pred = output.data.max(1, keepdim=True)[1]
    correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    total += target.shape[0]

    loss = criterion(output, target)
    total_loss += loss.item()
log = ('Test Epoch: {}\tAcc:[{}/{}]{:.6f} Loss: {:.6f}'.format(
            epoch, int(correct), int(total), float(correct)/float(total)*100, total_loss))
print(log)
    
    
    
    


Train Epoch: 0	Acc:[59849/59904]99.908186 Loss: 1.146383
Train Epoch: 1	Acc:[59865/59904]99.934896 Loss: 0.835427
Train Epoch: 2	Acc:[59892/59904]99.979968 Loss: 0.290450
Train Epoch: 3	Acc:[59903/59904]99.998331 Loss: 0.071935
Train Epoch: 4	Acc:[59850/59904]99.909856 Loss: 1.666681
Train Epoch: 5	Acc:[59855/59904]99.918202 Loss: 1.244281
Train Epoch: 6	Acc:[59870/59904]99.943243 Loss: 0.815544
Train Epoch: 7	Acc:[59894/59904]99.983307 Loss: 0.248720
Train Epoch: 8	Acc:[59872/59904]99.946581 Loss: 0.747108
Train Epoch: 9	Acc:[59882/59904]99.963275 Loss: 0.659531
Train Epoch: 10	Acc:[59904/59904]100.000000 Loss: 0.056610
Train Epoch: 11	Acc:[59902/59904]99.996661 Loss: 0.073902
Train Epoch: 12	Acc:[59838/59904]99.889824 Loss: 1.824219
Train Epoch: 13	Acc:[59884/59904]99.966613 Loss: 0.643098
Train Epoch: 14	Acc:[59891/59904]99.978299 Loss: 0.318978
Train Epoch: 15	Acc:[59875/59904]99.951589 Loss: 0.542340
Train Epoch: 16	Acc:[59846/59904]99.903178 Loss: 1.677997
Train Epoch: 17	Acc:[59

#### Q5:
Please print the training and testing accuracy.